In [65]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def table_scraper(station, start_date, end_date):
    current_date = start_date
    search_url = 'https://www.wunderground.com/history/daily/{}/date/{}-{}-{}'

    while current_date < end_date:
        format_search_url = search_url.format('K'+station, current_date.year, current_date.month, current_date.day)

        driver = webdriver.Chrome()
        driver.get(format_search_url)
        # Makes sure all tables are loaded
        table = WebDriverWait(driver,20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table")))[-1]

        # Reads the correct html table, and selects rows and cols of interest
        _df = pd.read_html(table.get_attribute('outerHTML'))[0].dropna()[["Time", "Condition"]]
        _df["Date"] = current_date

        if start_date == current_date:
            _df.to_csv(f'{station}_{start_date.year}_{end_date.year}.csv', header=True, index=False, mode="w")
        else:
            _df.to_csv(f'{station}_{start_date.year}_{end_date.year}.csv', header=False, index=False, mode="a")

        current_date += timedelta(days=1)


In [ ]:
location = 'GAI'
start = datetime(year=2015, month=2, day=1)
end = datetime(year=2016, month=2, day=1)

table_scraper(location, start, end)

In [167]:
def table_cleaning(table):
    _df = pd.read_csv(table)

    _df["Time"] = pd.to_datetime(_df["Date"] + " " + _df["Time"], format="%Y-%m-%d %I:%M %p").dt.floor('h')
    del _df["Date"]

    _df["Time"][_df["Time"].dt.hour == 0] += timedelta(days=1)

    _df = _df.groupby(["Time"])["Condition"].agg(pd.Series.mode).reset_index()

    rank = {"Fair": 1, "Partly Cloudy": 2, "Mostly Cloudy": 3, "Cloudy": 4, "Thunder": 5}
    for idx, row in _df.iterrows():
        if isinstance(_df["Condition"][idx], np.ndarray):
            _df.at[idx, "Condition"] = [con.replace(" / Windy", "").replace(" in the Vicinity", "") for con in _df["Condition"][idx]]
            _df.at[idx, "Condition"] = sorted(_df["Condition"][idx], key = lambda ele: rank[ele])[len(_df["Condition"][idx]) // 2]

        elif isinstance(_df["Condition"][idx], str):
            _df.at[idx, "Condition"] = _df["Condition"][idx].replace(" / Windy", "").replace(" in the Vicinity", "")

    _df.to_csv(table, header=True, index=False, mode="w")

In [169]:
table_cleaning("GAI_2015_2016.csv")

/var/folders/nt/w3ctvjn94jn89w856x4xh8zc0000gn/T/ipykernel_92951/1214078264.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df["Time"][_df["Time"].dt.hour == 0] += timedelta(days=1)
